<a href="https://colab.research.google.com/github/akitim/SkillFactory_current/blob/main/MATH%26ML_6_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%90%D0%BA%D0%B8%D0%BC%D0%BA%D0%B8%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.**

In [2]:
import numpy as np
import pandas as pd

path = 'https://raw.githubusercontent.com/akitim/datasets/main/Advertising.csv'
# path = '/content/Advertising.csv'
data = pd.read_csv(path, index_col=0)
data.sample(5)

,TV,radio,newspaper,sales
48,239.9,41.5,18.5,23.2
187,139.5,2.1,26.6,10.3
31,292.9,28.3,43.2,21.4
172,164.5,20.9,47.4,14.5
133,8.4,27.2,2.1,5.7


**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [3]:
print('Количество пропусков в признаках:')
data.isna().sum(axis=0)

Количество пропусков в признаках:


TV           0
radio        0
newspaper    0
sales        0
dtype: int64

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)**

In [4]:
target = 'sales'
y = data[target].values
X = data.drop(target, axis=1).values

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [5]:
import numpy as np

X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [6]:
# X = X / np.sqrt(np.sum(np.square(X), axis=0))
X = X / np.linalg.norm(X, axis=0)

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм координатного спуска для случая нормализованных данных:

**Задано:**

* $X=(x_{ij})$ - матрица наблюдений, размерностью $dim(X)=(n, m)$
* $N=1000$ - количество итераций

**Примечание:** *1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.*

**Алгоритм (математическая запись):**
* Создать нулевой вектор параметров $w_0=(0, 0,..., 0)^T$
* Для всех $t=1, 2, ..., N$ итераций:
    * Для всех $k = 1, 2,..., m$:
        * Фиксируем значение всех признаков, кроме $k$-ого и вычисляем прогноз модели линейной регрессии.Для этого исключаем признак $k$-ый из данных и $w_j$ из параметров при построении прогноза.
        Математически это можно записать следующим образом:

        $$h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j $$

        **Примечание:**
        
        *Обратите, что в данной записи текущий признак под номером $k$ не участвует в сумме.Сравните эту запись с классической записью прогноза линейной регрессии в случае нормированных данных (когда участвуют все признаки):*

        $$h_i = \sum_{j=1}^{m} x_{ij}w_{j}$$
        
        * Вычисляем новое значение параметра $k$-ого коэффициента:
        $$w_k = \sum_{i=1}^{n} x_{ik} (y_i - h_i) = x_k^T(y-h) $$

    * Вычисляем значение функции потерь и сохраняем в историю изменения функции потерь (В оценке функции потерь участвуют все признаки):
        $$\hat{y_i} = \sum_{j=1}^{m}x_{ij}w_j$$
        $$Loss_t = \frac{1}{n} \sum_{i=1}^{n}(y_i-\hat{y_i})^2$$
        
        или в векторном виде:
        
        $$\hat{y} = Xw$$
        $$Loss_t = \frac{1}{n}(y-\hat{y})^T(y-\hat{y})$$
    



**Алгоритм (псевдокод):**
```python

num_iters = #количество итераций
m = # количество строк в матрице X
n = # количество столбцов в матрице X
w = #вектор размера nx1, состояющий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum((X @ w) - y) ** 2)/(len(y))

```

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [61]:
import numpy as np

class CustomLinearRegression:

    def __init__(self, solver='CD_linsearch',
                 fit_intercept=False, max_iter=100, tol=1e-5):
        self.solver = solver
        self.fit_intercept = fit_intercept
        self.max_iter = max_iter
        self.tolerance = tol

    def CD_linsearch(self, X, y, w_0=None):
    #координатный спуск с минимизацией линейным поиском по каждой координате

        if self.fit_intercept:
            ones = np.ones(shape=(X.shape[0], 1))
            X = np.hstack(ones, X)

        n, m = X.shape
        w = w_0 if w_0 is not None else np.zeros(shape=(1, m))
        # w = w.reshape((-1, 1)) # для матричных операций

        # функция потерь
        def Loss(w, X, y):
            n = y.size
            return (1 / n * (X @ w.T - y).T @ (X @ w.T - y))[0, 0]

        L_curr = Loss(w, X, y)
        status = ''
        Ls = [L_curr]
        coords = [w.copy()]
        for i in range(self.max_iter):
            for k in range(m):
            # линейный поиск минимума в напр. w_k: делаем "большой" шаг в обе
            # стороны от т. w_0_k,чтобы гарантированно попасть в направление
            # убывания Loss смотрим, уменьшилась ли Loss-ф-ции в новых точках;
            # если да, переходим в т. с наименьшим зн-ем Loss, нет -
            # уменьшаем шаг в 1 / phi раз, снова смотрим Loss в новых точках

                step = 101 # шаг линейного поиска
                phi = 0.61803398875 # коэф. уменьшения шага линейного поиска
                cond = False
                while not cond:
                    # 'большой' шаг в обе стороны по коорд. w_k,

                    w_k_plus, w_k_minus = w[0, k] + step, w[0, k] - step

                    # вычисляем Loss
                    w_k_curr = w[0, k]
                    w[0, k] = w_k_plus
                    L_plus = Loss(w, X, y)
                    w[0, k] = w_k_minus
                    L_minus = Loss(w, X, y)
                    w[0, k] = w_k_curr

                    #debug
                    # print('i, k=', i, k)
                    # print('L_curr=', L_curr)
                    # print('L_plus=', L_plus)
                    # print('L_minus=', L_minus)
                    # print('step=', step)

                    #удалось ли хотя бы на одном из отр. и пол. шагов уменьшить Loss
                    cond = L_curr >= L_plus or L_curr >= L_minus

                    #если удалось
                    if cond:
                        w[0, k] = [w_k_minus, w_k_plus][np.argmin([L_minus, L_plus])]
                        L_curr = np.min([L_minus, L_plus])
                        coords += [w.copy()]

                    step *= phi

                Ls += [L_curr]

            # if np.linalg.norm(coords[-2] - coords[-1]) <= tol:
            if Ls[-2] - Ls[-1] <= self.tolerance:
                status = 'Success'
                break

            if i == self.max_iter - 1:
                status = 'Max_iter reached!'

        return {'status': status, 'weights': w[0], 'Ls': Ls, 'trajectory': coords,
                'nit': i + 1}

    def fit(self, X, y, w_0):

        if self.solver == 'CD_linsearch':
            res = self.CD_linsearch(X, y, self.fit_intercept, w_0,
                               self.max_iter, self.tolerance)
        elif self.solver == 'CD_matrix':
            res = None

        self.status = res['status']
        if self.fit_intercept:
            self._intercept = res['weights'][0]
            self.coef_ = res['weights'][1:]
        else:
            self._intercept = 0
            self.coef_ = res['weights']

        self.Ls = res['Ls']
        self.trajectory = res['trajectory']
        self.nit = res['nit']

        return None

    pass

# ==============================================================================
# ==============================================================================
def coord_descent_linsearch(X, y, w_0=None, max_iter=100, tol=1e-5):
    #координатный спуск с минимизацией линейным поиском по каждой координате

    n, m = X.shape
    w = w_0 if w_0 is not None else np.zeros(shape=(1, m))
    # w = w.reshape((-1, 1)) # для матричных операций

    # функция потерь
    def Loss(w, X, y):
        n = y.size
        return (1 / n * (X @ w.T - y).T @ (X @ w.T - y))[0, 0]

    L_curr = Loss(w, X, y)
    status = ''
    Ls = [L_curr]
    coords = [w.copy()]
    for i in range(max_iter):
        for k in range(m):
            # линейный поиск минимума в напр. w_k: делаем "большой" шаг в обе
            # стороны от т. w_0_k,чтобы гарантированно попасть в направление
            # убывания Loss смотрим, уменьшилась ли Loss-ф-ции в новых точках;
            # если да, переходим в т. с наименьшим зн-ем Loss, нет -
            # уменьшаем шаг в 1 / phi раз, снова смотрим Loss в новых точках

            step = 101 # шаг линейного поиска
            phi = 0.61803398875 # коэф. уменьшения шага линейного поиска
            cond = False
            while not cond:
                # 'большой' шаг в обе стороны по коорд. w_k,

                w_k_plus, w_k_minus = w[0, k] + step, w[0, k] - step

                # вычисляем Loss
                w_k_curr = w[0, k]
                w[0, k] = w_k_plus
                L_plus = Loss(w, X, y)
                w[0, k] = w_k_minus
                L_minus = Loss(w, X, y)
                w[0, k] = w_k_curr

                #debug
                # print('i, k=', i, k)
                # print('L_curr=', L_curr)
                # print('L_plus=', L_plus)
                # print('L_minus=', L_minus)
                # print('step=', step)

                #удалось ли хотя бы на одном из отр. и пол. шагов уменьшить Loss
                cond = L_curr >= L_plus or L_curr >= L_minus

                #если удалось
                if cond:
                    w[0, k] = [w_k_minus, w_k_plus][np.argmin([L_minus, L_plus])]
                    L_curr = np.min([L_minus, L_plus])
                    coords += [w.copy()]

                step *= phi

                # if step <= 1e-6: break
            Ls += [L_curr]

        # if np.linalg.norm(coords[-2] - coords[-1]) <= tol:
        if Ls[-2] - Ls[-1] <= tol:
            status = 'Success'
            break

        if i == max_iter - 1:
            status = 'Max_iter reached!'

    return {'status': status, 'weights': w[0], 'Ls': Ls, 'trajectory': coords,
            'nit': i + 1}

# ==============================================================================
# ==============================================================================
def coord_descent_analytic(X, y, fit_intercept=False, w_0=None, max_iter=100, tol=1e-5):

    #координатный спуск с оптимизацией линейным поиском по каждой координате
    if fit_intercept:
        ones = np.ones(shape=(X.shape[0], 1))
        X = np.hstack(ones, X)

    n, m = X.shape
    w = w_0 if w_0 is not None else np.zeros(shape=(1, m))
    # w = w.reshape((-1, 1)) # для матричных операций

    # функция потерь
    def Loss(w, X, y):
        n = y.size
        return (1 / n * (X @ w.T - y).T @ (X @ w.T - y))[0, 0]

    L_curr = Loss(w, X, y)
    status = ''
    Ls = [L_curr]
    # print(w)
    coords = [w.copy()] #sic!
    # coords.append(w)
    # print('coords init: ', coords)
    for i in range(max_iter):
        for k in range(m):
            w[0, k] = 0
            # w[0, k] = -2 * X[:, [k]].T @ (X @ w.T - y) / np.sum(X[:, k])
            w[0, k] = - X[:, [k]].T @ (X @ w.T - y)

            # print('coords before:', coords)
            coords.append(w.copy()) #sic!
            # print('coords after:', coords, '\n')
            L_curr = Loss(w, X, y)
            Ls += [L_curr]

        if Ls[-2] - Ls[-1] <= tol:
            status = 'Success'
            break

        if i == max_iter - 1:
            status = 'Max_iter reached!'

    return {'status': status, 'weights': w[0], 'Ls': Ls, 'trajectory': coords,
            'nit': i + 1}

# ==============================================================================
# ==============================================================================
w_0 = np.zeros(shape=(1, X.shape[1]))
res_cd_ls = coord_descent_linsearch(X, y, w_0=w_0, max_iter=100, tol=1e-10)
res_cd_an = coord_descent_analytic(X, y, w_0=w_0, max_iter=100, tol=1e-10)
print ('Коэффициенты модели c линейным поиском:',
       *np.round(res_cd_ls['weights'], 8))
print ('Коэффициенты модели с аналитической оптимизацией:',
       *np.round(res_cd_an['weights'], 8))

Коэффициенты модели c линейным поиском: 41.56406844 110.13127669 73.52778043 -0.55083171
Коэффициенты модели с аналитической оптимизацией: 41.56406844 110.13127669 73.52778043 -0.55083171


In [55]:
# delete
import matplotlib.pyplot as plt

trajectory_1 = np.asarray(res_cd_ls['trajectory'])[00:500]
n_iter = res_cd_ls['nit'] * X.shape[0]
print('n_iter =  ', n_iter)

k = 1
mask = [ind * k for ind in range(n_iter // k)] + [-1]

u = trajectory[:, 0, 1]
v = trajectory[:, 0, 2]
z = trajectory[:, 0, 3]

fig, ax = plt.subplots(figsize=(9, 9), subplot_kw={'projection':'3d'})
ax.plot(u, v, z, 'C0--');


NameError: ignored

In [ ]:
# delete
import matplotlib.pyplot as plt

Ls = res_cd_v1['Ls']
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(range(1, len(Ls) + 1), Ls);

ax.set_xscale('log')


In [14]:
# delete
def coord_descent_v2(X, y, fit_intercept=False, w_0=None, max_iter=100, tol=1e-5):

    #координатный спуск с оптимизацией линейным поиском по каждой координате
    if fit_intercept:
        ones = np.ones(shape=(X.shape[0], 1))
        X = np.hstack(ones, X)

    n, m = X.shape
    w = w_0 if w_0 is not None else np.zeros(shape=(1, m))
    # w = w.reshape((-1, 1)) # для матричных операций

    # функция потерь
    def Loss(w, X, y):
        n = y.size
        return (1 / n * (X @ w.T - y).T @ (X @ w.T - y))[0, 0]

    L_curr = Loss(w, X, y)
    status = ''
    Ls = [L_curr]
    print(w)
    coords = [w.copy()] #sic!
    # coords.append(w)
    print('coords init: ', coords)
    for i in range(max_iter):
        for k in range(m):
            w[0, k] = 0
            # w[0, k] = -2 * X[:, [k]].T @ (X @ w.T - y) / np.sum(X[:, k])
            w[0, k] = - X[:, [k]].T @ (X @ w.T - y)

            print('coords before:', coords)
            coords.append(w.copy()) #sic!
            print('coords after:', coords, '\n')
            L_curr = Loss(w, X, y)
            Ls += [L_curr]

        if Ls[-2] - Ls[-1] <= tol:
            status = 'Success'
            break

        if i == max_iter - 1:
            status = 'Max_iter reached!'

    return {'status': status, 'weights': w[0], 'Ls': Ls, 'trajectory': coords,
            'nit': i + 1}

res_cd_v2 = coord_descent_v2(X, y, fit_intercept=False, max_iter=100, tol=1e-10)
res_cd_v2['weights'], res_cd_v2['Ls']
res_cd_v2['trajectory'][0]

[[0. 0. 0. 0.]]
coords init:  [array([[0., 0., 0., 0.]])]
coords before: [array([[0., 0., 0., 0.]])]
coords after: [array([[0., 0., 0., 0.]]), array([[198.30809678,   0.        ,   0.        ,   0.        ]])] 

coords before: [array([[0., 0., 0., 0.]]), array([[198.30809678,   0.        ,   0.        ,   0.        ]])]
coords after: [array([[0., 0., 0., 0.]]), array([[198.30809678,   0.        ,   0.        ,   0.        ]]), array([[198.30809678,  28.97502365,   0.        ,   0.        ]])] 

coords before: [array([[0., 0., 0., 0.]]), array([[198.30809678,   0.        ,   0.        ,   0.        ]]), array([[198.30809678,  28.97502365,   0.        ,   0.        ]])]
coords after: [array([[0., 0., 0., 0.]]), array([[198.30809678,   0.        ,   0.        ,   0.        ]]), array([[198.30809678,  28.97502365,   0.        ,   0.        ]]), array([[198.30809678,  28.97502365,   1.22438981,   0.        ]])] 

coords before: [array([[0., 0., 0., 0.]]), array([[198.30809678,   0.        ,

array([[0., 0., 0., 0.]])

Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [44]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(X, y)

print(model.coef_)

[[ 41.56217205 110.13144155  73.52860638  -0.55006384]]


Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [ ]:
#ваш код

**Добавим единичный столбец**

In [ ]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [ ]:
#ваш код

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [ ]:
#ваш код

**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [ ]:
#ваш код

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла)
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [ ]:
#ваш код

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [ ]:
# ваш код

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [ ]:
# ваш код

**Выведите вектор весов, к которому сошелся метод.**

In [ ]:
# ваш код

**Выведите среднеквадратичную ошибку на последней итерации.**

In [ ]:
# ваш код